# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown
from openai import OpenAI

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [4]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

API key looks good so far


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
system_prompt = """You are individual that possesses a unique
and highly valuable combination of deep technical
expertise and excellent communication skills.
You  grasp complex, specialized concepts and then distill
them into simple, understandable terms for people without
the same technical background.

Present your answer as markdown
"""

In [7]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
    ]

In [8]:
# Get gpt-4o-mini to answer, with streaming

response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))



Certainly! Let's break down the code you've provided:

### Code Explanation

```python
yield from {book.get("author") for book in books if book.get("author")}
```

#### Components of the Code:

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}`: This part of the code is a **set comprehension**. It creates a set of authors by iterating through a collection called `books`.
   - `book.get("author")`: This method retrieves the value associated with the key `"author"` from each `book` dictionary. If the specified key does not exist, `None` is returned.
   - `if book.get("author")`: This condition filters out any authors that are `None` or any falsy value, ensuring that only valid authors are included in the set.

2. **Yielding Values**:
   - `yield from`: This keyword is used in Python to yield values from a generator or an iterable. In this context, it takes the set created from the set comprehension and yields each author one at a time.

### What the Code Does

- The code iterates over a collection of `books`, each of which is expected to be a dictionary. For each book, it checks if the `"author"` key exists and has a valid value (not `None` or falsy).
- It collects all unique authors into a set to ensure no duplicates are included. The use of a set here also makes looking up authors efficient.
- Finally, it yields each author one at a time, allowing the consuming code to process each author individually.

### Why Use This Code?

- **Unique Values**: Using a set guarantees that each author is only yielded once, even if multiple books have the same author.
- **Memory Efficiency**: Using `yield from` allows handling potentially large collections of authors efficiently without having to store them all in memory at once.
- **Ease of Use**: This pattern is useful for generating a stream of results where the number of authors is not known in advance, and the consumer can process each author as it arrives.

### Summary

This code is effective for extracting unique authors from a list of book dictionaries, yielding each author for further processing without duplicates and in an efficient manner.

In [9]:
# Get Llama 3.2 to answer

response = ollama.chat.completions.create(model=MODEL_LLAMA, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))

**Understanding Recursive Yield**

```python
yield from {book.get("author") for book in books if book.get("author")}
```
is a piece of Python code that utilizes the `yield from` syntax to delegate iterations over a sub-generators. This allows us to generate an iterable sequence one step at a time, avoiding memory overhead.

### Breakdown

*   `{book.get("author") for book in books if book.get("author")}` is a generator expression:
    *   It defines a small, anonymous function (lambda-like) that takes `books` as input.
    *   For each item `book` in `books`, it calls another function (`get`) to retrieve the `"author"` attribute from an object.
    *   The resulting values are generated on-the-fly and sent through the generator expression.
*   `yield from …`: The outer expression uses `yield from` to delegate control over iterations to the inner nested generator.
    *   Instead of dealing with each iterable individually, `yield from` sends its arguments down into the first argument, which it executes and yields values on.

**Equivalent Code**

Here is an equivalent version of this code using traditional loops:

```python
def book_authors(books):
    authors = []
    for book in books:
        if "author" in book:
            author_name = book["author"]
            yield from (author_name,)
            authors.append(author_name)

# To use this function with our recursive yield version 
# you can use it like so
for author_name in book_authors(books):
    # Use this here
```

### Why is Yield From useful?

In programming languages that natively don't support nested generator iterations (like CPython), using `yield from` allows for elegant implementation of cooperative multitasking. It also simplifies the logic of writing generators.

Note: However, all such recursive-yield constructions rely on Python’s behavior towards generator expressions in its specific use-case.